|Name|Contribution|
|--|--|
|Nayak Vinayak Vinod|100%|
|Ajith Praveen R|100%|
|Nayak Uttam Jnaneshwar Reshma|100%|

# Question 1


An application collects large quantities of machine generated data and stores it. The data collected are periodically analysed to generate critical systems warnings. Due to the sensitive nature of data, we would like to ensure as much availability as possible.

Assume that there are 8 servers in each rack, there are 16 such racks and each machine is having hard disk size of 4 GB. Block size is 128MB. How many data blocks can be stored if we use RF=3 and RF=4? 

**Solution**

Per Rack calculation

- Servers per rack = 8
- Size of one server = 4 GB
- Size of a block = 128 MB

With this info, we can say

$\text{Blocks per rack} = \frac{\text{Total Memory Per Rack}}{\text{Block Size}}$

$\therefore \text{Blocks per rack} = \frac{\text{Servers per rack x Size of one server}}{\text{Block  Size}}$

$\therefore \text{Blocks per rack} = \frac{8 \times 4 \text{ GB}}{128 \text{ MB}}$

$\therefore \text{Blocks per rack} = \frac{8 \times 4 \times 2^{30}}{128 \times 2^{20}}$

$\therefore \text{Blocks per rack} = 256$

Since we have 16 such racks, the total number of blocks of memory available to us are $16 \times 256 = 4096$ blocks.

$$
\pagebreak
$$

We are aware of the HDFS Rack awareness strategy which states the following

- There should not be more than 1 replica on the same Datanode.
- More then 2 replica’s of a single block is not allowed on the same Rack.
- The number of racks used inside a Hadoop cluster must be smaller than the number of replicas.

In accordance with the above guidelines and using a Replication factor of 3, we could come up with the following scheme.

![](rack_awareness_rf3.png)

- We can group racks into sets of 3. 
- Rack 1 can save 128 distinct blocks and a single copy of them on the same rack.
- Corresponding to these 128 blocks, one replica can be stored in Rack 2. This ensures that all these 128 blocks have an RF = 3.
- Rack 3 can save 128 distinct blocks with a single copy of them on the same rack.
- Corresponding to these 128 blocks on Rack3, one replica can be stored in Rack 2. This ensures that all these 128 blocks have an RF = 3.
- This means that between 3 racks, we can store a total of 128 + 128 = 256 blocks.
- Since we have 16 racks, and the closest multiple of 3 without going over is 5, we can have 5 groups of 3 racks each which store 256 * 5 = 1280 blocks and one rack would stay unutilized because we cannot store all 3 replicas of a block on the same server.

$$
\pagebreak
$$

Similarly, for a Replication factor of 4, we could argue as follows

![](rack_awareness_rf4.png)

- Group the racks in sets of 2.
- Consider Rack 1 and Rack 2. Once we store a block on Rack 1, we can store one replica of the same on Rack 1 and two replicas on Rack 2.
- This means that 256 / 2 = 128 blocks in all can be stored on a combination of 2 racks.
- Since we can group the 16 racks into 8 sets of 2 racks each, we can store 8 * 128 = 1024 blocks of data on the given hadoop cluster.    


**Answer**

For the given hadoop cluster with 

- *Replication Factor = 3, we can store a max of 1280 blocks but 256 blocks are unutilized*
- *Replication Factor = 4, we can store a max of 1024 blocks with the complete utilization of all blocks in the rack.*

$$
\pagebreak
$$

# Question 2

Use map reduce to compute the mean of a set of values stored in a list L.

**Solution**


*Pseudocode*

1. Load the data into memory in an array/list L
2. Define a function which which takes an index and maps the same to the value in the list L at that index
```python
def Lambda(x):
    return L[x]
# apply Lambda on a list of chronological indices spanning length of list L
mapped_list = [Lambda(i) for i in range(len(L))]
```
3. Use the binary operator sum to reduce the elements of list L into a single number i.e. summation of all the values in the list and eventually divide it by the number of elements in the list.
```python
# apply reduce sigma on Lambda(L) and divide it by number of elements
mean = sum(mapped_list) / len(mapped_list)
```

In [1]:
# Define the list of values whose mean needs to be computed
data = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3]

# Map stage
mapping_function = lambda x: data[x]
mapped_list = [mapping_function(x) for x in range(len(data))]
print(f"Mapped List: {mapped_list}")

# Reduce stage
mu = sum(mapped_list) / len(mapped_list)
print(f"Mean of all elements in the list: {mu}")

# Check if what we get is what is expected
assert mu == 3.9

Mapped List: [3, 1, 4, 1, 5, 9, 2, 6, 5, 3]
Mean of all elements in the list: 3.9


$$
\pagebreak
$$

## Question 3

### Solution

Using map-reduce, given a list of tuples L containing the actual and expected values, compute the mean square error.

$\text{MSE} = \frac{1}{n}\Sigma_{i=1}^{N}(Y_i - \hat{Y_i})^2$

where

- MSE: Mean Squared Error
- n: Number of data points
- $Y_i$: True target value
- $\hat{Y_i}$: Predicted target value 

*Pseudocode*

1. Load the data into memory in an array/list L
2. Define a function which which takes one tuple from the list and computes the square of the difference of it's contents
```python
def Lambda(t):
    true, pred = t[0], t[1]
    return (true - pred) ** 2
# apply Lambda on a list of chronological indices spanning length of list L
mapped_list = [Lambda(t) for t in L]
```
3. Use the binary operator sum to reduce the elements of mapped list into a single number i.e. summation of all the values in the list and eventually divide it by the number of elements in the list.
```python
# apply reduce sigma on Lambda(L) and divide it by number of elements
mean_squared_error = sum(mapped_list) / len(mapped_list)
```

In [2]:
true_values = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3]
predicted_values = [4, 0, 3, 5, 1, 2, 5, 4, 3, 1]

tuplized_list = list(zip(true_values, predicted_values))
print(f"Reformatted list: {tuplized_list}")

# Map the tuples to a squared diff function
def Lambda(t):
    true, pred = t[0], t[1]
    return (true - pred) ** 2
mapped_list = [Lambda(t) for t in tuplized_list]
print(f"Mapped list: {mapped_list}")

# apply reduce sigma on Lambda(L) and divide it by number of elements
mean_squared_error = sum(mapped_list) / len(mapped_list)
print(f"Mean squared error of elements in the tuplized list: {mean_squared_error}")

# Check the mean squared error should equal expected mean squared error
assert mean_squared_error == 10.5

Reformatted list: [(3, 4), (1, 0), (4, 3), (1, 5), (5, 1), (9, 2), (2, 5), (6, 4), (5, 3), (3, 1)]
Mapped list: [1, 1, 1, 16, 16, 49, 9, 4, 4, 4]
Mean squared error of elements in the tuplized list: 10.5
